In [2]:
# import libraries
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
# load the trained model
# model = load_model('model.keras')
model = load_model('model.h5')

# load the encoder pickle file
with open('label_en_gender.pkl', 'rb') as file:
    label_en_gender = pickle.load(file)

with open('onehot_en_geo.pkl', 'rb') as file:
    onehot_en_geo = pickle.load(file)

# load the scaler pickle file
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

2025-01-31 07:51:25.175831: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
# example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',    
    'Gender' : 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
# one hot encoding for Geography
geo_encoded = onehot_en_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_en_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/prime/Documents/ai-projects/myenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0

In [6]:
input_df=pd.DataFrame([input_data])
input_df

CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [7]:
# encode categorical variables
input_df['Gender'] = label_en_gender.transform(input_df['Gender'])
input_df

CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France       1   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [8]:
# contatinatetion one hot encoded
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0

In [9]:
# scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.52111599,  0.91601335,  0.10961719, -0.68538967, -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [10]:
# predict the output
pred = model.predict(input_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([[0.02975262]], dtype=float32)

In [11]:
prediction = pred[0][0]
prediction

np.float32(0.029752618)

In [12]:
if prediction > 0.5:
    print('The customer is leave the bank')
else:
    print('The customer is not leave the bank')

The customer is not leave the bank
